In [11]:
!pip install konlpy
!pip install gensim

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import numpy as np
import pandas as pd
import urllib.request
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences
# 데이터 불러오기
try:
    urllib.request.urlretrieve("https://raw.githubusercontent.com/tonykorea99/Spam-alart/main/OriginData2.csv", filename='OriginData2.csv')
    data = pd.read_csv('OriginData2.csv', encoding='utf-8')
    data = data.sample(frac=1, random_state=42)
    print('총 샘플 수:', len(data))
except Exception as e:
    print('에러 발생:', e)

# 필요한 열 선택 및 중복 제거
data = data[['v1', 'v2']]
data['v1'] = data['v1'].replace(['ham', 'spam'], [0, 1])
data.drop_duplicates(subset=['v2'], inplace=True)
print('중복 제거 후의 전체 샘플 수:', len(data))

# 형태소 분리
okt = Okt()

# 형태소 분리 및 토크나이징
data['tokens'] = data['v2'].apply(lambda x: okt.morphs(x))

# Doc2Vec 모델 학습을 위한 태그 설정
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(data['tokens'])]

# Doc2Vec 모델 학습
doc2vec_model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=100)
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

# 학습된 Doc2Vec 모델 저장
doc2vec_model.save("/content/drive/MyDrive/doc2vec_model")



<ipython-input-13-8d18e2ff3797>:14: DtypeWarning: Columns (2,3,4,5,6,10,11,15,16,20,21,24,25,26,30,31,35,36,40,41,45,46,50,51,55,56,60,61,65,66,70,71,75,76,80,81,85,86,90,91,95,96,100,101,105,106,110,111,115,116,120,121,125,126,130,131,135,136,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('OriginData2.csv', encoding='utf-8')


총 샘플 수: 87002
중복 제거 후의 전체 샘플 수: 84726


<ipython-input-13-8d18e2ff3797>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['v1'] = data['v1'].replace(['ham', 'spam'], [0, 1])
<ipython-input-13-8d18e2ff3797>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop_duplicates(subset=['v2'], inplace=True)


In [ ]:
# 문서를 벡터로 변환
X = [doc2vec_model.infer_vector(doc.words) for doc in tagged_data]

# 레이블 및 데이터 분리
y = data['v1'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# LSTM 모델 정의


model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=len(doc2vec_model.wv.index_to_key), output_dim=200, input_length=100))
model_lstm.add(Bidirectional(LSTM(128)))
model_lstm.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 문서 벡터에 패딩 추가
X_train_pad = pad_sequences(X_train)
X_test_pad = pad_sequences(X_test, maxlen=X_train_pad.shape[1])  # X_test의 패딩 길이를 X_train에 맞춤


In [16]:

# 모델 훈련
model_lstm.fit(X_train_pad, y_train, epochs=20, batch_size=64, validation_split=0.1)




Epoch 1/20
954/954 [==============================] - 25s 19ms/step - loss: 0.3822 - accuracy: 0.8677 - val_loss: 0.3828 - val_accuracy: 0.8677
Epoch 2/20
954/954 [==============================] - 15s 16ms/step - loss: 0.3599 - accuracy: 0.8756 - val_loss: 0.3433 - val_accuracy: 0.8753
Epoch 3/20
954/954 [==============================] - 15s 16ms/step - loss: 0.3403 - accuracy: 0.8786 - val_loss: 0.3355 - val_accuracy: 0.8770
Epoch 4/20
954/954 [==============================] - 15s 15ms/step - loss: 0.3347 - accuracy: 0.8815 - val_loss: 0.3324 - val_accuracy: 0.8793
Epoch 5/20
954/954 [==============================] - 15s 15ms/step - loss: 0.3304 - accuracy: 0.8846 - val_loss: 0.3311 - val_accuracy: 0.8789
Epoch 6/20
954/954 [==============================] - 15s 15ms/step - loss: 0.3263 - accuracy: 0.8862 - val_loss: 0.3285 - val_accuracy: 0.8858
Epoch 7/20
954/954 [==============================] - 16s 16ms/step - loss: 0.3247 - accuracy: 0.8871 - val_loss: 0.3289 - val_accuracy:

In [17]:
# 모델 평가
loss, accuracy = model_lstm.evaluate(X_test_pad, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# 모델 저장
model_path = '/content/drive/MyDrive/smishing_model.h5'
model_lstm.save(model_path)

530/530 [==============================] - 4s 7ms/step - loss: 0.3182 - accuracy: 0.8901
Loss: 0.31822940707206726, Accuracy: 0.890121579170227


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
